# Sportsbetting Project Bundesliga

This Notebook contains the development of a machine learning model to predict future Bundesliga games. The goal is to pass the algorithm two team names (i.e. Home Team and Away Team) to get the probability of the outcome of the game.

### Setup

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn

import tensorflow as tf
from tensorflow import keras

# fetching data from website 
import requests
import bs4
import lxml
from bs4 import BeautifulSoup

### Fetching Team market values from website
Team market value as a later feature 

In [2]:
req = requests.get('https://de.statista.com/statistik/daten/studie/216031/umfrage/marktwert-der-bundesligamannschaften/')
type(req)

requests.models.Response

In [3]:
# raw text
#req.text

In [4]:
soup = BeautifulSoup(req.text, 'html.parser')
#print(soup.prettify())

In [5]:
teams = soup.find('tbody').text
teams

'FC Bayern München \r891,4Borussia Dortmund\r615,2RasenBallsport Leipzig 552,68Bayer 04 Leverkusen\r343,45Borussia Mönchengladbach\r343,13Hertha BSC\r251,93TSG 1899 Hoffenheim\r238,33VfL Wolfsburg\r218,55Eintracht Frankfurt\r186,2FC Schalke 04\r165,8SC Freiburg 111,7SV Werder Bremen\r107,2FC Augsburg\r104,28VfB Stuttgart103,451. FC Köln101,81. FSV Mainz 0589,55Union Berlin60,6Arminia Bielefeld45,15'

In [6]:
# Remove unwanted characters and match teamnames with names used later on

teams_clean = teams.replace(' ', '')
teams_clean = teams_clean.replace('\r', '')
teams_clean = teams_clean.replace('FCBayernMünchen', 'BayernMunich')
teams_clean = teams_clean.replace('BorussiaDortmund', 'Dortmund')
teams_clean = teams_clean.replace('RasenBallsportLeipzig', 'RBLeipzig')
teams_clean = teams_clean.replace('BorussiaMönchengladbach', 'Gladbach')
teams_clean = teams_clean.replace('VfLWolfsburg', 'Wolfsburg')
teams_clean = teams_clean.replace('EintrachtFrankfurt', 'EinFrankfurt')
teams_clean = teams_clean.replace('HerthaBSC', 'Hertha')
teams_clean = teams_clean.replace('FCAugsburg', 'Augsburg')
teams_clean = teams_clean.replace('VfbSturrgart', 'Stuttgart')
teams_clean = teams_clean.replace('SCFreiburg', 'Freiburg')
teams_clean = teams_clean.replace('FCKöln', 'FCKoln')
teams_clean = teams_clean.replace('Bayer04Leverkusen', 'Leverkusen')
teams_clean = teams_clean.replace('FSVMainz05', 'Mainz')
teams_clean = teams_clean.replace('ArminiaBielefeld', 'Bielefeld')
teams_clean = teams_clean.replace('FCSchalke04', 'Schalke')
teams_clean = teams_clean.replace('TSG1899Hoffenheim', 'Hoffenheim')
teams_clean = teams_clean.replace('SVWerderBremen', 'WerderBremen')
teams_clean

'BayernMunich891,4Dortmund615,2RBLeipzig552,68Leverkusen343,45Gladbach343,13Hertha251,93Hoffenheim238,33Wolfsburg218,55EinFrankfurt186,2Schalke165,8Freiburg111,7WerderBremen107,2Augsburg104,28VfBStuttgart103,451.FCKoln101,81.Mainz89,55UnionBerlin60,6Bielefeld45,15'

In [7]:
# Split the string into alphabetic and numeric values
from itertools import groupby 

def split_text(s):
    for k, g in groupby(s, str.isalpha):
        yield ''.join(g)

teamsdone = (list(split_text(teams_clean)))
print(teamsdone)

['BayernMunich', '891,4', 'Dortmund', '615,2', 'RBLeipzig', '552,68', 'Leverkusen', '343,45', 'Gladbach', '343,13', 'Hertha', '251,93', 'Hoffenheim', '238,33', 'Wolfsburg', '218,55', 'EinFrankfurt', '186,2', 'Schalke', '165,8', 'Freiburg', '111,7', 'WerderBremen', '107,2', 'Augsburg', '104,28', 'VfBStuttgart', '103,451.', 'FCKoln', '101,81.', 'Mainz', '89,55', 'UnionBerlin', '60,6', 'Bielefeld', '45,15']


In [8]:
# Convert list into dictionary to bind team names to values 
def Convert(lst):
    res_dct = {lst[i]: lst[i + 1] for i in range(0, len(lst), 2)}
    return res_dct

teamsdone = Convert(teamsdone)
teamsdone

{'BayernMunich': '891,4',
 'Dortmund': '615,2',
 'RBLeipzig': '552,68',
 'Leverkusen': '343,45',
 'Gladbach': '343,13',
 'Hertha': '251,93',
 'Hoffenheim': '238,33',
 'Wolfsburg': '218,55',
 'EinFrankfurt': '186,2',
 'Schalke': '165,8',
 'Freiburg': '111,7',
 'WerderBremen': '107,2',
 'Augsburg': '104,28',
 'VfBStuttgart': '103,451.',
 'FCKoln': '101,81.',
 'Mainz': '89,55',
 'UnionBerlin': '60,6',
 'Bielefeld': '45,15'}

In [9]:
sortedTeams = dict(sorted(teamsdone.items())) #Sort alphabetically, like we will do later on too
sortedTeams

{'Augsburg': '104,28',
 'BayernMunich': '891,4',
 'Bielefeld': '45,15',
 'Dortmund': '615,2',
 'EinFrankfurt': '186,2',
 'FCKoln': '101,81.',
 'Freiburg': '111,7',
 'Gladbach': '343,13',
 'Hertha': '251,93',
 'Hoffenheim': '238,33',
 'Leverkusen': '343,45',
 'Mainz': '89,55',
 'RBLeipzig': '552,68',
 'Schalke': '165,8',
 'UnionBerlin': '60,6',
 'VfBStuttgart': '103,451.',
 'WerderBremen': '107,2',
 'Wolfsburg': '218,55'}

These values will be used later on in this project as a new feature. After Evaluating the process, it would have been easier to create a simple Excel sheet and update the values from time to time. However, now we have automated the process and don't need to spend more time with this feature.

# Load Data

In [10]:
data1 = pd.read_csv('D1.csv')
data1.tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
148,D1,20/01/2021,17:30,Schalke 04,FC Koln,1,2,A,0,1,...,1.66,0.00,2.13,1.81,2.14,1.81,2.14,1.92,2.07,1.81
149,D1,20/01/2021,19:30,Augsburg,Bayern Munich,0,1,A,0,1,...,4.43,2.25,2.07,1.86,2.10,1.81,2.13,2.01,2.02,1.84
150,D1,20/01/2021,19:30,Bielefeld,Stuttgart,3,0,H,1,0,...,2.10,0.50,2.03,1.90,2.00,1.93,2.11,1.93,2.01,1.87
151,D1,20/01/2021,19:30,Freiburg,Ein Frankfurt,2,2,D,1,1,...,2.28,0.25,1.84,2.09,1.86,2.07,1.86,2.17,1.78,2.10
152,D1,20/01/2021,19:30,RB Leipzig,Union Berlin,1,0,H,0,0,...,2.08,-1.00,1.90,2.03,1.91,2.02,2.00,2.06,1.91,1.95


In [11]:
data2 = pd.read_csv('MD_10.csv', skiprows=1)
data2.head()

,Team,Rank,P,M,W,D,L,G,GA,GD,...,GD.1,Rank.2,P.2,M.2,W.2,D.2,L.2,G.2,GA.2,GD.2
0,Bayern Munich,1,23,10,7,2,1,34,16,18,...,14,2,12,5,4,0,1,13,9,4
1,Bayer Leverkusen,2,22,10,6,4,0,19,9,10,...,3,1,14,6,4,2,0,11,4,7
2,RB Leipzig,3,21,10,6,3,1,21,9,12,...,11,9,6,5,1,3,1,7,6,1
3,Borussia Dortmund,4,19,10,6,1,3,22,10,12,...,8,4,10,5,3,1,1,9,5,4
4,VfL Wolfsburg,5,18,10,4,6,0,16,10,6,...,4,7,7,5,1,4,0,7,5,2


As of first, we will take care of the training features **X** included in the datasets. This includes the selection of important features. The goal is to use as many important features to create a row vector for each team with their according stats.

In [12]:
X1 = data1.iloc[:, 3:15] # Leaving only most relevant features
X1.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST
0,Bayern Munich,Schalke 04,8,0,H,3,0,H,22,5,12,1
1,Ein Frankfurt,Bielefeld,1,1,D,0,0,D,18,14,6,4
2,FC Koln,Hoffenheim,2,3,A,1,2,A,13,13,6,7
3,Stuttgart,Freiburg,2,3,A,0,2,A,22,7,7,6
4,Union Berlin,Augsburg,1,3,A,0,1,A,13,9,3,5


### Abbreviation Definition

- **FTHG** = Full time home team goals  $\qquad\qquad$  **FTAG** = Full time away team goals 
- **FTR** = Full time result            $\qquad\qquad\qquad\qquad\;\;$  **HTR** = Half time result           
- **HTHG** = Half time home team goals  $\qquad\quad\;\;\;$  **HTAG** = Half time away team goals 
- **HS** = Home team shots              $\qquad\qquad\qquad\;\;\;\;\;\;\;$  **AS** = Away team shots             
- **HST** = Home team shots on target   $\qquad\quad\;\;\;\;\;$  **AST** Away team shots on target         

In [13]:
X1_clean = X1.drop(['HTR'], axis=1) # Dropping half time result for training data
X1_clean[0:10]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST
0,Bayern Munich,Schalke 04,8,0,H,3,0,22,5,12,1
1,Ein Frankfurt,Bielefeld,1,1,D,0,0,18,14,6,4
2,FC Koln,Hoffenheim,2,3,A,1,2,13,13,6,7
3,Stuttgart,Freiburg,2,3,A,0,2,22,7,7,6
4,Union Berlin,Augsburg,1,3,A,0,1,13,9,3,5
5,Werder Bremen,Hertha,1,4,A,0,2,17,13,7,6
6,Dortmund,M'gladbach,3,0,H,1,0,9,8,4,2
7,RB Leipzig,Mainz,3,1,H,2,0,23,8,10,1
8,Wolfsburg,Leverkusen,0,0,D,0,0,9,6,1,2
9,Hertha,Ein Frankfurt,1,3,A,0,2,12,10,6,3


In [14]:
X2_clean = data2.iloc[:, 0:9] # Leaving only important features
X2_clean[0:10]

,Team,Rank,P,M,W,D,L,G,GA
0,Bayern Munich,1,23,10,7,2,1,34,16
1,Bayer Leverkusen,2,22,10,6,4,0,19,9
2,RB Leipzig,3,21,10,6,3,1,21,9
3,Borussia Dortmund,4,19,10,6,1,3,22,10
4,VfL Wolfsburg,5,18,10,4,6,0,16,10
5,Union Berlin,6,16,10,4,4,2,22,14
6,M&ouml;nchengladbach,7,16,10,4,4,2,19,16
7,VfB Stuttgart,8,14,10,3,5,2,19,16
8,Eintracht Frankfurt,9,13,10,2,7,1,15,17
9,1899 Hoffenheim,10,12,10,3,3,4,18,17


### Integrate both Datasets
In this section, we will write a function that will combine the two datasets and sorts the features respectively. The Idea is that each Team has it's own row, including their rank, goals scored etc. as well as the stats of the last games

In [15]:
# First, we need to make sure the team names of both datasets match

for name in range(len(X2_clean['Team'])):
    if (X2_clean.iloc[name, 0] == '1. FC K&ouml;ln'):
        X2_clean.iloc[name, 0] = 'FC Koln'
    if (X2_clean.iloc[name, 0] == 'VfL Wolfsburg'):
        X2_clean.iloc[name, 0] = 'Wolfsburg'
    if (X2_clean.iloc[name, 0] == 'M&ouml;nchengladbach'):
        X2_clean.iloc[name, 0] = 'Gladbach'
    if (X2_clean.iloc[name, 0] == 'Borussia Dortmund'):
        X2_clean.iloc[name, 0] = 'Dortmund'
    if (X2_clean.iloc[name, 0] == 'VfB Stuttgart'):
        X2_clean.iloc[name, 0] = 'Stuttgart'
    if (X2_clean.iloc[name, 0] == 'Eintracht Frankfurt'):
        X2_clean.iloc[name, 0] = 'Ein Frankfurt'
    if (X2_clean.iloc[name, 0] == '1899 Hoffenheim'):
        X2_clean.iloc[name, 0] = 'Hoffenheim'
    if (X2_clean.iloc[name, 0] == 'FC Augsburg'):
        X2_clean.iloc[name, 0] = 'Augsburg'
    if (X2_clean.iloc[name, 0] == 'Hertha BSC Berlin'):
        X2_clean.iloc[name, 0] = 'Hertha'
    if (X2_clean.iloc[name, 0] == 'SC Freiburg'):
        X2_clean.iloc[name, 0] = 'Freiburg'
    if (X2_clean.iloc[name, 0] == 'Arminia Bielefeld'):
        X2_clean.iloc[name, 0] = 'Bielefeld'
    if (X2_clean.iloc[name, 0] == 'FSV Mainz 05'):
        X2_clean.iloc[name, 0] = 'Mainz'
    if (X2_clean.iloc[name, 0] == 'FC Schalke 04'):
        X2_clean.iloc[name, 0] = 'Schalke'
    if (X2_clean.iloc[name, 0] == 'Bayer Leverkusen'):
        X2_clean.iloc[name, 0] = 'Leverkusen'
        
X2_clean['Team']

0     Bayern Munich
1        Leverkusen
2        RB Leipzig
3          Dortmund
4         Wolfsburg
5      Union Berlin
6          Gladbach
7         Stuttgart
8     Ein Frankfurt
9        Hoffenheim
10         Augsburg
11           Hertha
12    Werder Bremen
13         Freiburg
14          FC Koln
15        Bielefeld
16            Mainz
17          Schalke
Name: Team, dtype: object

In [16]:
X1_clean = X1_clean.replace('M\'gladbach', 'Gladbach') # Need to make one change on this Dataset too
X1_clean = X1_clean.replace('Schalke 04', 'Schalke')
X1_clean[0:7]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST
0,Bayern Munich,Schalke,8,0,H,3,0,22,5,12,1
1,Ein Frankfurt,Bielefeld,1,1,D,0,0,18,14,6,4
2,FC Koln,Hoffenheim,2,3,A,1,2,13,13,6,7
3,Stuttgart,Freiburg,2,3,A,0,2,22,7,7,6
4,Union Berlin,Augsburg,1,3,A,0,1,13,9,3,5
5,Werder Bremen,Hertha,1,4,A,0,2,17,13,7,6
6,Dortmund,Gladbach,3,0,H,1,0,9,8,4,2


In [17]:
# Converting Panda dataframe to Numpy Array (maybe needed later)
X1_clean_num = X1_clean.iloc[:,:].values # using numpy array
X1_clean_num[0:5]

array([['Bayern Munich', 'Schalke', 8, 0, 'H', 3, 0, 22, 5, 12, 1],
       ['Ein Frankfurt', 'Bielefeld', 1, 1, 'D', 0, 0, 18, 14, 6, 4],
       ['FC Koln', 'Hoffenheim', 2, 3, 'A', 1, 2, 13, 13, 6, 7],
       ['Stuttgart', 'Freiburg', 2, 3, 'A', 0, 2, 22, 7, 7, 6],
       ['Union Berlin', 'Augsburg', 1, 3, 'A', 0, 1, 13, 9, 3, 5]],
      dtype=object)

### Determining Home Team stats
Since the Home and Away performances differ drastically for some teams, we will divide these performances in the following code to HomeTeam stats and AwayTeam stats. Then, we will take the average.

In [18]:
X1_clean.head()

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HS,AS,HST,AST
0,Bayern Munich,Schalke,8,0,H,3,0,22,5,12,1
1,Ein Frankfurt,Bielefeld,1,1,D,0,0,18,14,6,4
2,FC Koln,Hoffenheim,2,3,A,1,2,13,13,6,7
3,Stuttgart,Freiburg,2,3,A,0,2,22,7,7,6
4,Union Berlin,Augsburg,1,3,A,0,1,13,9,3,5


##### Total home team stats for each Bundesliga team

In [19]:
teamnames = X2_clean['Team']
sum_HGS = {key: 0 for key in teamnames} #Initialize dictionary (Team : HG) and initialize values to 0
sum_HGC = {key: 0 for key in teamnames} #Initialize dictionary (Team : HGC) and initialize values to 0
sum_HST = {key: 0 for key in teamnames} #Initialize dictionary (Team : HST) and initialize values to 0
sum_HSTC = {key: 0 for key in teamnames} #Initialize dictionary (Team : HST) and initialize values to 0
sum_HS = {key: 0 for key in teamnames} #Initialize dictionary (Team : HS) and initialize values to 0
sum_HSC = {key: 0 for key in teamnames} #Initialize dictionary (Team : HS) and initialize values to 0

hometeams = X1_clean['HomeTeam']
HGS = X1_clean['FTHG']
HGC = X1_clean['FTAG']
HST = X1_clean['HST']
HSTC = X1_clean['AST']
HS = X1_clean['HS']
HSC = X1_clean['AS']

homegames_count = {key: 0 for key in teamnames} # Counting total home games for each team to calculate the average later on

# Loop through all the matches and add up the HST for each team
for team in range(len(hometeams)): 
        sum_HGS[hometeams[team]] = sum_HGS[hometeams[team]] + HGS[team] # Hometeam goals scored
        sum_HGC[hometeams[team]] = sum_HGC[hometeams[team]] + HGC[team] # Hometeam goals conceded
        sum_HST[hometeams[team]] = sum_HST[hometeams[team]] + HST[team] # Hometeam shots on target
        sum_HSTC[hometeams[team]] = sum_HSTC[hometeams[team]] + HSTC[team] # Hometeam shots on target conceded
        sum_HS[hometeams[team]] = sum_HS[hometeams[team]] + HS[team] # Hometeam shots
        sum_HSC[hometeams[team]] = sum_HSC[hometeams[team]] + HSC[team] # Hometeam shots conceded
        homegames_count[hometeams[team]] = homegames_count[hometeams[team]] + 1 # Increasing count of homegames by 1
        #print(hometeams[team], sum_HST[hometeams[team]])

print("Total goals Home Team: ")
print(sum_HGS)
print("\n")
print("Total goals conceded Home Team: ")
print(sum_HGC)
print("\n")
print("Total shots on Target at home for each Team: ")
print(sum_HST)
print("\n")
print("Total shots on Target conceded at home for each Team: ")
print(sum_HSTC)
print("\n")
print("Total shots at home for each Team: ")
print(sum_HS)
print("\n")
print("Total shots conceded at home for each Team: ")
print(sum_HSC)
print("\n")
print("Total Home games for each team: ")
print(homegames_count)

Total goals Home Team: 
{'Bayern Munich': 30, 'Leverkusen': 16, 'RB Leipzig': 18, 'Dortmund': 17, 'Wolfsburg': 14, 'Union Berlin': 20, 'Gladbach': 14, 'Stuttgart': 11, 'Ein Frankfurt': 14, 'Hoffenheim': 12, 'Augsburg': 9, 'Hertha': 10, 'Werder Bremen': 8, 'Freiburg': 20, 'FC Koln': 8, 'Bielefeld': 9, 'Mainz': 6, 'Schalke': 8}


Total goals conceded Home Team: 
{'Bayern Munich': 11, 'Leverkusen': 10, 'RB Leipzig': 6, 'Dortmund': 11, 'Wolfsburg': 8, 'Union Berlin': 11, 'Gladbach': 9, 'Stuttgart': 15, 'Ein Frankfurt': 10, 'Hoffenheim': 13, 'Augsburg': 16, 'Hertha': 15, 'Werder Bremen': 12, 'Freiburg': 14, 'FC Koln': 18, 'Bielefeld': 11, 'Mainz': 17, 'Schalke': 15}


Total shots on Target at home for each Team: 
{'Bayern Munich': 63, 'Leverkusen': 45, 'RB Leipzig': 61, 'Dortmund': 45, 'Wolfsburg': 39, 'Union Berlin': 51, 'Gladbach': 45, 'Stuttgart': 36, 'Ein Frankfurt': 40, 'Hoffenheim': 45, 'Augsburg': 29, 'Hertha': 37, 'Werder Bremen': 29, 'Freiburg': 55, 'FC Koln': 28, 'Bielefeld': 23, 

##### Average stats per game as a home team for each Bundesliga team

In [20]:
avg_HGS = {key: 0 for key in teamnames} #Initialize dictionary (Team : average FTHG)
avg_HGC = {key: 0 for key in teamnames} #Initialize dictionary (Team : average FTHG)
avg_HST = {key: 0 for key in teamnames} #Initialize dictionary (Team : average HST)
avg_HSTC = {key: 0 for key in teamnames} #Initialize dictionary (Team : average HST)
avg_HS = {key: 0 for key in teamnames} #Initialize dictionary (Team : average HS)
avg_HSC = {key: 0 for key in teamnames} #Initialize dictionary (Team : average HS)

for team in range(len(teamnames)):
    avg_HGS[teamnames[team]] = sum_HGS[teamnames[team]] / homegames_count[teamnames[team]]
    avg_HGC[teamnames[team]] = sum_HGC[teamnames[team]] / homegames_count[teamnames[team]]
    avg_HST[teamnames[team]] = sum_HST[teamnames[team]] / homegames_count[teamnames[team]]
    avg_HSTC[teamnames[team]] = sum_HSTC[teamnames[team]] / homegames_count[teamnames[team]]
    avg_HS[teamnames[team]] = sum_HS[teamnames[team]] / homegames_count[teamnames[team]]
    avg_HSC[teamnames[team]] = sum_HSC[teamnames[team]] / homegames_count[teamnames[team]]

print("Average goals scored as a Home team")
rounded_HGS = {key : round(avg_HGS[key], 2) for key in avg_HGS} # Round numbers
print(rounded_HGS)
print('\n')
print("Average goals conceded as a Home team")
rounded_HGC = {key : round(avg_HGC[key], 2) for key in avg_HGC} # Round numbers
print(rounded_HGC)
print('\n')
print("Average shots on target as a Home team")
rounded_HST = {key : round(avg_HST[key], 2) for key in avg_HST} # Round numbers
print(rounded_HST)
print('\n')
print("Average shots on target conceded as a Home team")
rounded_HSTC = {key : round(avg_HSTC[key], 2) for key in avg_HSTC} # Round numbers
print(rounded_HSTC)
print('\n')
print("Average shots as a Home team")
rounded_HS = {key : round(avg_HS[key], 2) for key in avg_HS} # Round numbers
print(rounded_HS)
print('\n')
print("Average shots conceded as a Home team")
rounded_HSC = {key : round(avg_HSC[key], 2) for key in avg_HSC} # Round numbers
print(rounded_HSC)

Average goals scored as a Home team
{'Bayern Munich': 3.75, 'Leverkusen': 2.0, 'RB Leipzig': 2.0, 'Dortmund': 2.12, 'Wolfsburg': 1.75, 'Union Berlin': 2.22, 'Gladbach': 1.56, 'Stuttgart': 1.38, 'Ein Frankfurt': 1.75, 'Hoffenheim': 1.5, 'Augsburg': 1.0, 'Hertha': 1.25, 'Werder Bremen': 1.0, 'Freiburg': 2.22, 'FC Koln': 0.89, 'Bielefeld': 1.0, 'Mainz': 0.67, 'Schalke': 0.89}


Average goals conceded as a Home team
{'Bayern Munich': 1.38, 'Leverkusen': 1.25, 'RB Leipzig': 0.67, 'Dortmund': 1.38, 'Wolfsburg': 1.0, 'Union Berlin': 1.22, 'Gladbach': 1.0, 'Stuttgart': 1.88, 'Ein Frankfurt': 1.25, 'Hoffenheim': 1.62, 'Augsburg': 1.78, 'Hertha': 1.88, 'Werder Bremen': 1.5, 'Freiburg': 1.56, 'FC Koln': 2.0, 'Bielefeld': 1.22, 'Mainz': 1.89, 'Schalke': 1.67}


Average shots on target as a Home team
{'Bayern Munich': 7.88, 'Leverkusen': 5.62, 'RB Leipzig': 6.78, 'Dortmund': 5.62, 'Wolfsburg': 4.88, 'Union Berlin': 5.67, 'Gladbach': 5.0, 'Stuttgart': 4.5, 'Ein Frankfurt': 5.0, 'Hoffenheim': 5.62, '

Below is a more convenient and shorter way of coding that represents the same findings as above, without creating a dictionary, but a panda dataframe for a cleaner look. However, the above presented dictionary can be used as a double-check to make sure that each statistic has the right value.

In [21]:
StatsHome = pd.DataFrame(columns=('Team','HGS','HGC','HST','HSTC','HS','HSC'))

TeamHome = X1_clean.groupby('HomeTeam')
StatsHome.Team = sorted(teamnames) # Sort alphabetically, take care that statistics match with team
StatsHome.HGS = TeamHome.FTHG.mean().values
StatsHome.HGC = TeamHome.FTAG.mean().values
StatsHome.HST = TeamHome.HST.mean().values
StatsHome.HSTC = TeamHome.AST.mean().values
StatsHome.HS = TeamHome.HS.mean().values
StatsHome.HSC = TeamHome.AS.mean().values
StatsHome = StatsHome.round(2)
StatsHome.head()

,Team,HGS,HGC,HST,HSTC,HS,HSC
0,Augsburg,1.00,1.78,3.22,5.89,10.22,15.00
1,Bayern Munich,3.75,1.38,7.88,4.12,18.25,9.62
2,Bielefeld,1.00,1.22,2.56,5.44,8.33,13.89
3,Dortmund,2.12,1.38,5.62,4.38,14.25,11.00
4,Ein Frankfurt,1.75,1.25,5.00,3.38,13.25,10.38


### Calculating Away Team stats
In the following code we sum up the total stats (goals, shots, ..) for each Awayteam and calculate the average for each feature

In [22]:
sum_AGS = {key: 0 for key in teamnames} #Initialize dictionary (Team : AG) and set values to 0
sum_AGC = {key: 0 for key in teamnames} #Initialize dictionary (Team : AGC) and set values to 0
sum_AST = {key: 0 for key in teamnames} #Initialize dictionary (Team : AST) and set values to 0
sum_ASTC = {key: 0 for key in teamnames} #Initialize dictionary (Team : ASTC) and set values to 0
sum_AS = {key: 0 for key in teamnames} #Initialize dictionary (Team : AS) and set values to 0
sum_ASC = {key: 0 for key in teamnames} #Initialize dictionary (Team : ASC) and set values to 0

awayteams = X1_clean['AwayTeam']
AGS = X1_clean['FTAG']
AGC = X1_clean['FTHG']
AST = X1_clean['AST']
ASTC = X1_clean['HST']
AS = X1_clean['AS']
ASC = X1_clean['HS']

awaygames_count = {key: 0 for key in teamnames} # Counting total away games for each team to calculate the average later on

# Loop through all matches and add up the AST for each team
for team in range(len(awayteams)): 
        sum_AGS[awayteams[team]] = sum_AGS[awayteams[team]] + AGS[team] # Awayteam goals scored
        sum_AGC[awayteams[team]] = sum_AGC[awayteams[team]] + AGC[team] # Awayteam goals conceded
        sum_AST[awayteams[team]] = sum_AST[awayteams[team]] + AST[team] # Awayteam shots on target
        sum_ASTC[awayteams[team]] = sum_ASTC[awayteams[team]] + ASTC[team] # Awayteam shots on target conceded
        sum_AS[awayteams[team]] = sum_AS[awayteams[team]] + AS[team] # Awayteam shots
        sum_ASC[awayteams[team]] = sum_ASC[awayteams[team]] + ASC[team] # Awayteam shots conceded
        awaygames_count[awayteams[team]] = awaygames_count[awayteams[team]] + 1

print("Total goals scored away for each Team: ")
print(sum_AGS)
print("\n")
print("Total goals conceded away for each Team: ")
print(sum_AGC)
print("\n")
print("Total shots on Target away for each Team: ")
print(sum_AST)
print("\n")
print("Total shots on Target conceded away for each Team: ")
print(sum_ASTC)
print("\n")
print("Total shots away for each Team: ")
print(sum_AS)
print("\n")
print("Total shots conceded away for each Team: ")
print(sum_ASC)
print("\n")
print("Total Away games for each team: ")
print(awaygames_count)

Total goals scored away for each Team: 
{'Bayern Munich': 19, 'Leverkusen': 16, 'RB Leipzig': 11, 'Dortmund': 16, 'Wolfsburg': 12, 'Union Berlin': 12, 'Gladbach': 17, 'Stuttgart': 21, 'Ein Frankfurt': 16, 'Hoffenheim': 13, 'Augsburg': 8, 'Hertha': 13, 'Werder Bremen': 11, 'Freiburg': 11, 'FC Koln': 7, 'Bielefeld': 4, 'Mainz': 9, 'Schalke': 6}


Total goals conceded away for each Team: 
{'Bayern Munich': 14, 'Leverkusen': 7, 'RB Leipzig': 8, 'Dortmund': 11, 'Wolfsburg': 11, 'Union Berlin': 10, 'Gladbach': 17, 'Stuttgart': 12, 'Ein Frankfurt': 16, 'Hoffenheim': 17, 'Augsburg': 10, 'Hertha': 13, 'Werder Bremen': 13, 'Freiburg': 14, 'FC Koln': 10, 'Bielefeld': 13, 'Mainz': 19, 'Schalke': 29}


Total shots on Target away for each Team: 
{'Bayern Munich': 47, 'Leverkusen': 46, 'RB Leipzig': 39, 'Dortmund': 58, 'Wolfsburg': 44, 'Union Berlin': 28, 'Gladbach': 51, 'Stuttgart': 55, 'Ein Frankfurt': 43, 'Hoffenheim': 47, 'Augsburg': 25, 'Hertha': 38, 'Werder Bremen': 36, 'Freiburg': 32, 'FC Koln

##### Average shots on target per game as an away team for each Bundesliga team

In [23]:
avg_AGS = {key: 0 for key in teamnames} #Initialize dictionary (Team : average AGS)
avg_AGC = {key: 0 for key in teamnames} #Initialize dictionary (Team : average AGC)
avg_AST = {key: 0 for key in teamnames} #Initialize dictionary (Team : average AST)
avg_ASTC = {key: 0 for key in teamnames} #Initialize dictionary (Team : average ASTC)
avg_AS = {key: 0 for key in teamnames} #Initialize dictionary (Team : average AS)
avg_ASC = {key: 0 for key in teamnames} #Initialize dictionary (Team : average ASC)


for team in range(len(teamnames)):
    avg_AGS[teamnames[team]] = sum_AGS[teamnames[team]] / awaygames_count[teamnames[team]]
    avg_AGC[teamnames[team]] = sum_AGC[teamnames[team]] / awaygames_count[teamnames[team]]
    avg_AST[teamnames[team]] = sum_AST[teamnames[team]] / awaygames_count[teamnames[team]]
    avg_ASTC[teamnames[team]] = sum_ASTC[teamnames[team]] / awaygames_count[teamnames[team]]
    avg_AS[teamnames[team]] = sum_AS[teamnames[team]] / awaygames_count[teamnames[team]]
    avg_ASC[teamnames[team]] = sum_ASC[teamnames[team]] / awaygames_count[teamnames[team]]
    
print("Average goals scored as an Away team")
rounded_AGS = {key : round(avg_AGS[key], 2) for key in avg_AGS} # Round numbers
print(rounded_AGS)
print('\n')
print("Average goals conceded as an Away team")
rounded_AGC = {key : round(avg_AGC[key], 2) for key in avg_AGC} # Round numbers
print(rounded_AGC)
print('\n')
print("Average shots on target as an Away team")
rounded_AST = {key : round(avg_AST[key], 2) for key in avg_AST} # Round numbers
print(rounded_AST)
print('\n')
print("Average shots on target conceded as an Away team")
rounded_ASTC = {key : round(avg_ASTC[key], 2) for key in avg_ASTC} # Round numbers
print(rounded_ASTC)
print('\n')
print("Average shots as an Away team")
rounded_AS = {key : round(avg_AS[key], 2) for key in avg_AS} # Round numbers
print(rounded_AS)
print('\n')
print("Average shots conceded as an Away team")
rounded_ASC = {key : round(avg_ASC[key], 2) for key in avg_ASC} # Round numbers
print(rounded_ASC)

Average goals scored as an Away team
{'Bayern Munich': 2.11, 'Leverkusen': 1.78, 'RB Leipzig': 1.38, 'Dortmund': 1.78, 'Wolfsburg': 1.33, 'Union Berlin': 1.5, 'Gladbach': 2.12, 'Stuttgart': 2.33, 'Ein Frankfurt': 1.78, 'Hoffenheim': 1.44, 'Augsburg': 1.0, 'Hertha': 1.44, 'Werder Bremen': 1.22, 'Freiburg': 1.38, 'FC Koln': 0.88, 'Bielefeld': 0.5, 'Mainz': 1.12, 'Schalke': 0.75}


Average goals conceded as an Away team
{'Bayern Munich': 1.56, 'Leverkusen': 0.78, 'RB Leipzig': 1.0, 'Dortmund': 1.22, 'Wolfsburg': 1.22, 'Union Berlin': 1.25, 'Gladbach': 2.12, 'Stuttgart': 1.33, 'Ein Frankfurt': 1.78, 'Hoffenheim': 1.89, 'Augsburg': 1.25, 'Hertha': 1.44, 'Werder Bremen': 1.44, 'Freiburg': 1.75, 'FC Koln': 1.25, 'Bielefeld': 1.62, 'Mainz': 2.38, 'Schalke': 3.62}


Average shots on target as an Away team
{'Bayern Munich': 5.22, 'Leverkusen': 5.11, 'RB Leipzig': 4.88, 'Dortmund': 6.44, 'Wolfsburg': 4.89, 'Union Berlin': 3.5, 'Gladbach': 6.38, 'Stuttgart': 6.11, 'Ein Frankfurt': 4.78, 'Hoffenhei

Again, a shorter way to visualize each teams individual performances

In [24]:
StatsAway = pd.DataFrame(columns=('Team','AGS','AGC','AST','ASTC','AS','ASC'))

TeamAway = X1_clean.groupby('AwayTeam')
StatsAway.Team = sorted(teamnames) # Sort alphabetically, take care that statistics match with team
StatsAway.AGS = TeamAway.FTAG.mean().values
StatsAway.AGC = TeamAway.FTHG.mean().values
StatsAway.AST = TeamAway.AST.mean().values
StatsAway.ASTC = TeamAway.HST.mean().values
StatsAway.AS = TeamAway.AS.mean().values
StatsAway.ASC = TeamAway.HS.mean().values
StatsAway = StatsAway.round(2)
StatsAway.head()

,Team,AGS,AGC,AST,ASTC,AS,ASC
0,Augsburg,1.00,1.25,3.12,4.75,9.12,13.00
1,Bayern Munich,2.11,1.56,5.22,4.11,14.22,11.67
2,Bielefeld,0.50,1.62,2.50,5.50,8.88,12.62
3,Dortmund,1.78,1.22,6.44,4.22,16.00,10.11
4,Ein Frankfurt,1.78,1.78,4.78,5.44,12.89,13.56


## Include Team Statistics to Matchday 
Now, that we have the Home and Away performances gathered together for each team, we will create a feature table that includes the games of each matchgame and includes the specific performances for the home and away team

In [25]:
FeatureTable = X1_clean[['HomeTeam', 'AwayTeam', 'FTR']]
FeatureTable

,HomeTeam,AwayTeam,FTR
0,Bayern Munich,Schalke,H
1,Ein Frankfurt,Bielefeld,D
2,FC Koln,Hoffenheim,A
3,Stuttgart,Freiburg,A
4,Union Berlin,Augsburg,A
...,...,...,...
148,Schalke,FC Koln,A
149,Augsburg,Bayern Munich,A
150,Bielefeld,Stuttgart,H
151,Freiburg,Ein Frankfurt,D


In [26]:
def AddHomeTeamStats (Matches, StatsHome):
    ''' Takes the two column Matchday Table (HomeTeam | AwayTeam) and attaches the Home Team stats'''
    
    f_HGS = [] #HomeTeam goals
    f_HGC = [] #HomeTeam goals conceded
    f_HST = [] #HomeTeam shots on target
    f_HSTC = [] # "" conceded
    f_HS = [] #HomeTeam shots
    f_HSC = [] # "" conceded

    for index, row in Matches.iterrows():
        # For each Match, loop trough the HomeTeam stats and attach the stats for the appropriate HomeTeam
        f_HGS.append(StatsHome[StatsHome['Team'] == row['HomeTeam']]['HGS'].values[0])
        f_HGC.append(StatsHome[StatsHome['Team'] == row['HomeTeam']]['HGC'].values[0])
        f_HST.append(StatsHome[StatsHome['Team'] == row['HomeTeam']]['HST'].values[0])
        f_HSTC.append(StatsHome[StatsHome['Team'] == row['HomeTeam']]['HSTC'].values[0])
        f_HS.append(StatsHome[StatsHome['Team'] == row['HomeTeam']]['HS'].values[0])
        f_HSC.append(StatsHome[StatsHome['Team'] == row['HomeTeam']]['HSC'].values[0])
        
        Matches['HGS'] = pd.Series(f_HGS) # Numpy Array to pd series to be able to attach to pandas df (Matches)
        Matches['HGC'] = pd.Series(f_HGC)
        Matches['HST'] = pd.Series(f_HST)
        Matches['HSTC'] = pd.Series(f_HSTC)
        Matches['HS'] = pd.Series(f_HS)
        Matches['HSC'] = pd.Series(f_HSC)

  #  print(Matches.head())

In [27]:
def AddAwayTeamStats(Matches, StatsAway):
    ''' Takes the two column Matchday Table (HomeTeam | AwayTeam) and attaches the Away Team stats'''
    
    f_AGS = []
    f_AGC = []
    f_AST = []
    f_ASTC = []
    f_AS = []
    f_ASC = []

    for index,row in Matches.iterrows():
        f_AGS.append(StatsAway[StatsAway['Team'] == row['AwayTeam']]['AGS'].values[0])
        f_AGC.append(StatsAway[StatsAway['Team'] == row['AwayTeam']]['AGC'].values[0])
        f_AST.append(StatsAway[StatsAway['Team'] == row['AwayTeam']]['AST'].values[0])
        f_ASTC.append(StatsAway[StatsAway['Team'] == row['AwayTeam']]['ASTC'].values[0])
        f_AS.append(StatsAway[StatsAway['Team'] == row['AwayTeam']]['AS'].values[0])
        f_ASC.append(StatsAway[StatsAway['Team'] == row['AwayTeam']]['ASC'].values[0])

        Matches['AGS'] = pd.Series(f_AGS)
        Matches['AGC'] = pd.Series(f_AGC)
        Matches['AST'] = pd.Series(f_AST)
        Matches['ASTC'] = pd.Series(f_ASTC)
        Matches['AS'] = pd.Series(f_AS)
        Matches['ASC'] = pd.Series(f_ASC)
        
   # print(Matches.head())

In [28]:
AddHomeTeamStats(FeatureTable, StatsHome)

In [29]:
FeatureTable.head()

,HomeTeam,AwayTeam,FTR,HGS,HGC,HST,HSTC,HS,HSC
0,Bayern Munich,Schalke,H,3.75,1.38,7.88,4.12,18.25,9.62
1,Ein Frankfurt,Bielefeld,D,1.75,1.25,5.00,3.38,13.25,10.38
2,FC Koln,Hoffenheim,A,0.89,2.00,3.11,5.56,9.11,13.11
3,Stuttgart,Freiburg,A,1.38,1.88,4.50,6.38,14.38,13.75
4,Union Berlin,Augsburg,A,2.22,1.22,5.67,3.67,13.33,10.33


In [30]:
AddAwayTeamStats(FeatureTable, StatsAway)

In [31]:
FeatureTable.head()

,HomeTeam,AwayTeam,FTR,HGS,HGC,HST,HSTC,HS,HSC,AGS,AGC,AST,ASTC,AS,ASC
0,Bayern Munich,Schalke,H,3.75,1.38,7.88,4.12,18.25,9.62,0.75,3.62,2.00,8.38,9.62,17.62
1,Ein Frankfurt,Bielefeld,D,1.75,1.25,5.00,3.38,13.25,10.38,0.50,1.62,2.50,5.50,8.88,12.62
2,FC Koln,Hoffenheim,A,0.89,2.00,3.11,5.56,9.11,13.11,1.44,1.89,5.22,5.11,13.22,12.33
3,Stuttgart,Freiburg,A,1.38,1.88,4.50,6.38,14.38,13.75,1.38,1.75,4.00,5.88,8.12,16.62
4,Union Berlin,Augsburg,A,2.22,1.22,5.67,3.67,13.33,10.33,1.00,1.25,3.12,4.75,9.12,13.00


#### Using Ordinal Encoder instead of one Hot Encoder to ensure a one-dimensional target shape

In [32]:
#from sklearn.preprocessing import OrdinalEncoder
#
#ordinal_encoder = OrdinalEncoder()
#y_encoded = ordinal_encoder.fit_transform(y)
#y_encoded[0:10] # Home win = 2, Draw = 1, Away win = 0

In [33]:
def transformResult(row):
    '''Converts results (H,A or D) into numeric values'''
    if(row.FTR == 'H'): # Win HomeTeam = 1
        return 1
    elif(row.FTR == 'A'): # Win AwayTeam = -1
        return -1
    else:
        return 0 # Draw = 0

In [34]:
FeatureTable["FTR"] = FeatureTable.apply(lambda row: transformResult(row),axis=1)
FeatureTable.tail(10)

,HomeTeam,AwayTeam,FTR,HGS,HGC,HST,HSTC,HS,HSC,AGS,AGC,AST,ASTC,AS,ASC
143,Ein Frankfurt,Schalke,1,1.75,1.25,5.00,3.38,13.25,10.38,0.75,3.62,2.00,8.38,9.62,17.62
144,Gladbach,Werder Bremen,1,1.56,1.00,5.00,3.44,13.00,12.89,1.22,1.44,4.00,4.33,9.78,13.78
145,Hertha,Hoffenheim,-1,1.25,1.88,4.62,3.50,13.38,9.12,1.44,1.89,5.22,5.11,13.22,12.33
146,Leverkusen,Dortmund,1,2.00,1.25,5.62,3.12,13.88,9.62,1.78,1.22,6.44,4.22,16.00,10.11
147,Mainz,Wolfsburg,-1,0.67,1.89,2.89,5.89,10.56,13.44,1.33,1.22,4.89,4.22,14.00,13.22
148,Schalke,FC Koln,-1,0.89,1.67,4.44,6.00,12.00,15.67,0.88,1.25,3.12,4.88,8.88,13.00
149,Augsburg,Bayern Munich,-1,1.00,1.78,3.22,5.89,10.22,15.00,2.11,1.56,5.22,4.11,14.22,11.67
150,Bielefeld,Stuttgart,1,1.00,1.22,2.56,5.44,8.33,13.89,2.33,1.33,6.11,4.00,15.22,12.33
151,Freiburg,Ein Frankfurt,0,2.22,1.56,6.11,3.89,16.11,12.22,1.78,1.78,4.78,5.44,12.89,13.56
152,RB Leipzig,Union Berlin,1,2.00,0.67,6.78,2.44,17.67,8.22,1.50,1.25,3.50,4.75,9.88,11.25


In [35]:
X = FeatureTable.drop(['HomeTeam', 'AwayTeam', 'FTR'], axis=1) # Dropping Team names and FTR
y = FeatureTable['FTR']

In [36]:
X.tail(10)

,HGS,HGC,HST,HSTC,HS,HSC,AGS,AGC,AST,ASTC,AS,ASC
143,1.75,1.25,5.00,3.38,13.25,10.38,0.75,3.62,2.00,8.38,9.62,17.62
144,1.56,1.00,5.00,3.44,13.00,12.89,1.22,1.44,4.00,4.33,9.78,13.78
145,1.25,1.88,4.62,3.50,13.38,9.12,1.44,1.89,5.22,5.11,13.22,12.33
146,2.00,1.25,5.62,3.12,13.88,9.62,1.78,1.22,6.44,4.22,16.00,10.11
147,0.67,1.89,2.89,5.89,10.56,13.44,1.33,1.22,4.89,4.22,14.00,13.22
148,0.89,1.67,4.44,6.00,12.00,15.67,0.88,1.25,3.12,4.88,8.88,13.00
149,1.00,1.78,3.22,5.89,10.22,15.00,2.11,1.56,5.22,4.11,14.22,11.67
150,1.00,1.22,2.56,5.44,8.33,13.89,2.33,1.33,6.11,4.00,15.22,12.33
151,2.22,1.56,6.11,3.89,16.11,12.22,1.78,1.78,4.78,5.44,12.89,13.56
152,2.00,0.67,6.78,2.44,17.67,8.22,1.50,1.25,3.50,4.75,9.88,11.25


In [37]:
y.head()

0    1
1    0
2   -1
3   -1
4   -1
Name: FTR, dtype: int64

### Dividing Data into Train and Test Data
The Test data will always be the most recent Matchday(s)

In [38]:
X_test = X.iloc[-9:, :] #Last Matchday (9 games) as test data
y_test = y.iloc[-9:]

X_train = X.iloc[:-9, :] # Rest of Dataset as training data
y_train = y.iloc[:-9]

#### Scaling the Data

In [39]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Training a Logistic regression Model for game prediction

In [40]:
from sklearn.linear_model import LogisticRegression

reg_clf = LogisticRegression(random_state=42)
reg_clf.fit(X_train, y_train)

LogisticRegression(random_state=42)

### Evaluation of training model

In [41]:
## After further research, cross validation is not valid to use in this application since it shuffles the matches randomly
#from sklearn.model_selection import cross_val_score
#cross_val_score(reg_clf, X_clean, y_encoded, cv=3, scoring='accuracy')

In [42]:
y_pred = reg_clf.predict(X_test)
y_pred

array([ 0,  0,  1, -1, -1, -1, -1,  0,  1])

### Evaluation score of Prediction from Logistic Regression model

In [43]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='micro')

0.6666666666666666

In [44]:
result = FeatureTable.iloc[-9:, 0:3]
result['Pred'] = y_pred
result

,HomeTeam,AwayTeam,FTR,Pred
144,Gladbach,Werder Bremen,1,0
145,Hertha,Hoffenheim,-1,0
146,Leverkusen,Dortmund,1,1
147,Mainz,Wolfsburg,-1,-1
148,Schalke,FC Koln,-1,-1
149,Augsburg,Bayern Munich,-1,-1
150,Bielefeld,Stuttgart,1,-1
151,Freiburg,Ein Frankfurt,0,0
152,RB Leipzig,Union Berlin,1,1


As we see, for this particular Matchday the Evaluation score was at 66%. The plan from here on is to include more features to see if we can increase the accuracy. If we can increase the accury for the Logistic regression model, we assume that we increase the accuracy in general, even if other models are used. Using other models, like the XGBoost and NN, would be the following step. 

## Predicting future matches
In the following future games will be predicted. With that being said, we don't know the outcome yet, as we did with the test set in the previous step. Only the teams of the next matchday will be implemented and fed into the model. The predicted resuklt will be compared with the actual result later on. As of now, a simple excel sheet containing all the matches of the next matchday will be created.

In [45]:
matchPred = pd.read_csv('MatchPred.csv')
matchPred

,HomeTeam,AwayTeam
0,Gladbach,Dortmund
1,Freiburg,Stuttgart
2,Mainz,RB Leipzig
3,Leverkusen,Wolfsburg
4,Augsburg,Union Berlin
5,Bielefeld,Ein Frankfurt
6,Hertha,Werder Bremen
7,Schalke,Bayern Munich
8,Hoffenheim,FC Koln


In [46]:
AddHomeTeamStats(matchPred, StatsHome)

In [47]:
AddAwayTeamStats(matchPred, StatsAway)

In [48]:
matchPred

,HomeTeam,AwayTeam,HGS,HGC,HST,HSTC,HS,HSC,AGS,AGC,AST,ASTC,AS,ASC
0,Gladbach,Dortmund,1.56,1.00,5.00,3.44,13.00,12.89,1.78,1.22,6.44,4.22,16.00,10.11
1,Freiburg,Stuttgart,2.22,1.56,6.11,3.89,16.11,12.22,2.33,1.33,6.11,4.00,15.22,12.33
2,Mainz,RB Leipzig,0.67,1.89,2.89,5.89,10.56,13.44,1.38,1.00,4.88,3.12,11.75,8.12
3,Leverkusen,Wolfsburg,2.00,1.25,5.62,3.12,13.88,9.62,1.33,1.22,4.89,4.22,14.00,13.22
4,Augsburg,Union Berlin,1.00,1.78,3.22,5.89,10.22,15.00,1.50,1.25,3.50,4.75,9.88,11.25
5,Bielefeld,Ein Frankfurt,1.00,1.22,2.56,5.44,8.33,13.89,1.78,1.78,4.78,5.44,12.89,13.56
6,Hertha,Werder Bremen,1.25,1.88,4.62,3.50,13.38,9.12,1.22,1.44,4.00,4.33,9.78,13.78
7,Schalke,Bayern Munich,0.89,1.67,4.44,6.00,12.00,15.67,2.11,1.56,5.22,4.11,14.22,11.67
8,Hoffenheim,FC Koln,1.50,1.62,5.62,4.25,11.12,11.25,0.88,1.25,3.12,4.88,8.88,13.00


In [58]:
X_pred = matchPred.drop(['HomeTeam', 'AwayTeam'], axis=1) # Dropping Team names and FTR
X_pred

,HGS,HGC,HST,HSTC,HS,HSC,AGS,AGC,AST,ASTC,AS,ASC
0,1.56,1.00,5.00,3.44,13.00,12.89,1.78,1.22,6.44,4.22,16.00,10.11
1,2.22,1.56,6.11,3.89,16.11,12.22,2.33,1.33,6.11,4.00,15.22,12.33
2,0.67,1.89,2.89,5.89,10.56,13.44,1.38,1.00,4.88,3.12,11.75,8.12
3,2.00,1.25,5.62,3.12,13.88,9.62,1.33,1.22,4.89,4.22,14.00,13.22
4,1.00,1.78,3.22,5.89,10.22,15.00,1.50,1.25,3.50,4.75,9.88,11.25
5,1.00,1.22,2.56,5.44,8.33,13.89,1.78,1.78,4.78,5.44,12.89,13.56
6,1.25,1.88,4.62,3.50,13.38,9.12,1.22,1.44,4.00,4.33,9.78,13.78
7,0.89,1.67,4.44,6.00,12.00,15.67,2.11,1.56,5.22,4.11,14.22,11.67
8,1.50,1.62,5.62,4.25,11.12,11.25,0.88,1.25,3.12,4.88,8.88,13.00


In [59]:
# scalar object already implemented previously (StandardScaler)
X_scaled = scaler.fit_transform(X)
X_pred = scaler.transform(X_pred) # Neglect this step and all predictions will be 0, important to include

In [60]:
#using full Dataset now (X) to train the model
# logistic Regression Model already implemented
reg_clf2 = LogisticRegression(random_state=42)
reg_clf2.fit(X_scaled, y)

LogisticRegression(random_state=42)

In [61]:
pred = reg_clf2.predict(X_pred)
pred

array([ 1,  0, -1,  1, -1,  1,  0, -1,  0])

In [62]:
# Visualization of Matchday Predictions
predict = matchPred.iloc[:, 0:2]
predict['Pred'] = pred
predict

,HomeTeam,AwayTeam,Pred
0,Gladbach,Dortmund,1
1,Freiburg,Stuttgart,0
2,Mainz,RB Leipzig,-1
3,Leverkusen,Wolfsburg,1
4,Augsburg,Union Berlin,-1
5,Bielefeld,Ein Frankfurt,1
6,Hertha,Werder Bremen,0
7,Schalke,Bayern Munich,-1
8,Hoffenheim,FC Koln,0


# Train a NN for Game predictions

In [54]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(X_train.shape)),
    keras.layers.Dense(10, activation="sigmoid"),
    keras.layers.Dense(10, activation="softmax")
])

In [55]:
model